In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go

# output to notebook
init_notebook_mode(True)

## Generating the Data and Visualizing It

The data is basically random gaussian noise, combined with a logarthmic trend, and a cosine-waveform of seasonality with a period of 12 units. In other words, you can think of this data as representing some time series over ten years with yearly seasonality.

In [2]:
x = np.arange(120)
com1 = np.random.rand(120)*.5
com2 = np.log(np.arange(0,2,2/120)+1)
com3 = np.array([np.cos(2*np.pi*np.arange(12)/12)*.5 for i in range(10)]).reshape(120,)
ts = com1+com2+com3

level = go.Scatter(
    x=x,
    y=com1
)
trend = go.Scatter(
    x=x,
    y=com2
)
seasonality = go.Scatter(
    x=x,
    y=com3
)
lev_trd = go.Scatter(
    x=x,
    y=com1+com2
)
trd_sea = go.Scatter(
    x=x,
    y=com2+com3
)
lev_trd_sea = go.Scatter(
    x=x,
    y=com1+com2+com3
)

fig = tools.make_subplots(rows=2, cols=3)

fig.append_trace(level, 1, 1)
fig.append_trace(trend, 1, 2)
fig.append_trace(seasonality, 1, 3)
fig.append_trace(lev_trd, 2, 1)
fig.append_trace(trd_sea, 2, 2)
fig.append_trace(lev_trd_sea, 2, 3)

iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]



## Functions

The first is the original function from Gregory Trubetskoy's blog.  I use this as a point of comparison.  Our functions vary a bit in terms of functionality.  His returns the original data that's been smoothed as well the next step prediction.  Mine only returns the next prediction.

To compare the effectiveness of vectorized code and numba, I wrote a for-loop implementation as well.  Lastly, I created a wrapper for these functions called `single_exp_smooth`.  These functions are not nested because numba behaves strangely when nestedt

In [ ]:
# https://grisha.org/blog/2016/01/29/triple-exponential-smoothing-forecasting/ 
# the final value of the output is equivalent to the SES prediction.
def exp_smooth(series, alpha):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha*series[n] + (1 - alpha)*result[n-1])
    return result

In [215]:
def ses(series,a):
    pred = 0
    for j in range(series.shape[0]):
        pred += a * series[length-(j+1)] * ((1-a) **j)
    return pred

def ses_b(series, a):
    length = series.shape[0]
    exp_ = (1-a) ** np.arange(length-1, -1, -1)
    return a * np.dot(series, exp_)

@jit(nopython=True)
def ses_nb(series, a):
    length = series.shape[0]
    exp_ = (1-a) ** np.arange(length-1, -1, -1)
    return a * np.dot(series, exp_)

@jit(nopython=True)
def ses_dn(series,a):
    pred = 0
    for j in range(series.shape[0]):
        pred += a * series[length-(j+1)] * ((1-a) **j)
    return pred

def single_exp_smooth(series, a, method='default'):
    ''' Single Exponential Smoothing with For Loop
    Input: Numpy array, alpha, number of predictions
    Output: Original array with steps number of predictions
    Does not take advantage of vectorized code, naive implementation.
    Notice that this implementation requires two loops.
    Assumes that elements at end of array (on the right) are most recent.
    [Beginning, ..., End]
    '''    
    if 'ndarray' not in str(type(series)):
        print('Please use a numpy array')

    elif method == 'default':
        return ses(series,a)
    
    elif method == 'broadcast':
        return ses_b(series,a)
    
    elif method == 'numba':
        return ses_nb(series,a)

In [220]:
%timeit exp_smooth(ts, .5)

44.1 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [221]:
%timeit single_exp_smooth(ts, .5, 'default')

48 µs ± 772 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [222]:
%timeit single_exp_smooth(ts, .5, 'broadcast')

5.21 µs ± 70.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [223]:
%timeit single_exp_smooth(ts, .5, 'numba')

2.13 µs ± 36.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
